<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/293_HITL_DataLoading_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data loading utilities for HITL Orchestrator

In [ ]:
"""Data loading utilities for HITL Orchestrator"""

import json
from pathlib import Path
from typing import List, Dict, Any, Optional


def load_tasks(data_dir: str, tasks_file: str) -> List[Dict[str, Any]]:
    """
    Load tasks from JSON file.

    Args:
        data_dir: Directory containing data files
        tasks_file: Name of tasks file

    Returns:
        List of task dictionaries
    """
    file_path = Path(data_dir) / tasks_file
    with open(file_path, 'r') as f:
        return json.load(f)


def load_agent_outputs(data_dir: str, agent_outputs_file: str) -> List[Dict[str, Any]]:
    """
    Load agent outputs from JSON file.

    Args:
        data_dir: Directory containing data files
        agent_outputs_file: Name of agent outputs file

    Returns:
        List of agent output dictionaries
    """
    file_path = Path(data_dir) / agent_outputs_file
    with open(file_path, 'r') as f:
        return json.load(f)


def load_routing_policy(data_dir: str, routing_policy_file: str) -> Dict[str, Any]:
    """
    Load routing policy from JSON file.

    Args:
        data_dir: Directory containing data files
        routing_policy_file: Name of routing policy file

    Returns:
        Routing policy dictionary
    """
    file_path = Path(data_dir) / routing_policy_file
    with open(file_path, 'r') as f:
        return json.load(f)


def load_human_reviews(data_dir: str, human_reviews_file: str) -> List[Dict[str, Any]]:
    """
    Load existing human reviews from JSON file (optional).

    Args:
        data_dir: Directory containing data files
        human_reviews_file: Name of human reviews file

    Returns:
        List of human review dictionaries (empty list if file doesn't exist)
    """
    file_path = Path(data_dir) / human_reviews_file
    if not file_path.exists():
        return []

    with open(file_path, 'r') as f:
        return json.load(f)


def load_audit_logs(data_dir: str, audit_logs_file: str) -> List[Dict[str, Any]]:
    """
    Load existing audit logs from JSON file (optional).

    Args:
        data_dir: Directory containing data files
        audit_logs_file: Name of audit logs file

    Returns:
        List of audit log dictionaries (empty list if file doesn't exist)
    """
    file_path = Path(data_dir) / audit_logs_file
    if not file_path.exists():
        return []

    with open(file_path, 'r') as f:
        return json.load(f)


def build_task_output_lookup(
    tasks: List[Dict[str, Any]],
    agent_outputs: List[Dict[str, Any]]
) -> Dict[str, Dict[str, Any]]:
    """
    Build lookup dictionary mapping task_id to combined task + output data.

    Args:
        tasks: List of tasks
        agent_outputs: List of agent outputs

    Returns:
        Dictionary mapping task_id to combined task and output data
    """
    lookup = {}

    # Create task lookup
    task_lookup = {task["task_id"]: task for task in tasks}

    # Create output lookup
    output_lookup = {output["task_id"]: output for output in agent_outputs}

    # Combine
    for task_id in task_lookup.keys():
        task_data = task_lookup[task_id].copy()
        output_data = output_lookup.get(task_id, {})

        lookup[task_id] = {
            "task": task_data,
            "agent_output": output_data.get("agent_output", {}),
            "confidence_score": output_data.get("confidence_score", 0.0)
        }

    return lookup




This part **is** pretty straightforward, and that’s a *good thing*. I’ll keep this **short, high-level, and conceptual**.

---

# 🧠 Big Picture: What This File Is For

This file is the **“loader dock”** for your orchestrator.

Its job is simple:

> **Take data from files on disk and turn it into clean, usable memory for the agent.**

No decisions.
No AI.
No logic.

Just **getting the ingredients ready** before cooking.

---

# 📦 Part 1: Loading Data from Files

All these functions follow the same pattern:

```python
load_tasks
load_agent_outputs
load_routing_policy
load_human_reviews
load_audit_logs
```

### Conceptually:

Each one answers a question like:

> “Hey orchestrator, can you go read this file and give me the data inside?”

They:

1. Find the file
2. Open it
3. Read the JSON
4. Hand back Python dictionaries/lists

Think of them as:

* librarians
* not judges
* not decision-makers

---

## 🔍 Why separate loaders?

Because each dataset has a **different role** in the system:

* tasks = work to do
* agent outputs = proposed answers
* routing policy = rules
* human reviews = feedback
* audit logs = history

Separating them:

* keeps things clean
* makes testing easier
* avoids mixing responsibilities

This is **good system hygiene**.

---

## 🛡️ Why some loaders are “optional”

```python
if not file_path.exists():
    return []
```

This is subtle but important.

It means:

* The system can start **fresh**
* Or resume from **past runs**
* Without crashing

That’s how real systems survive restarts.

---

# 🔗 Part 2: `build_task_output_lookup`

## 🧠 “Making the Agent Fast and Organized”

This is the **only function here that does anything slightly clever**.

### What problem it solves

Right now you have:

* a list of tasks
* a list of agent outputs

But your orchestrator keeps asking:

> “For THIS task, what did the agent say?”

Searching lists over and over is slow and messy.

---

### What this function does (in human terms)

1. **Indexes tasks by ID**
2. **Indexes agent outputs by ID**
3. **Stitches them together**
4. **Stores them in one place**

So now you can instantly say:

> “Give me everything about task_003.”

---

### The result looks like this (conceptually)

```text
task_id → {
  task info,
  agent output,
  confidence score
}
```

This is **preparing the ground** for:

* routing decisions
* human review packets
* audit logs

---

# 🎯 Why This Matters (Conceptually)

This file teaches an important lesson:

> **Smart agents aren’t smart because they “think harder” —
> they’re smart because their data is organized.**

You’re designing:

* clarity
* reliability
* speed
* debuggability

That’s real engineering.

---

# ✅ Big Takeaway

This module:

* loads facts
* avoids assumptions
* prepares clean inputs
* keeps the orchestrator honest

It’s intentionally boring — and that’s exactly why it’s correct.

